In [1]:
import sys
from os import path
import requests
from bs4 import BeautifulSoup
import csv

from requests.models import Response



In [2]:
website_url = requests.get("https://en.wikipedia.org/wiki/Opinion_polling_for_the_2021_German_federal_election").text

soup = BeautifulSoup(website_url, "html.parser")
type(soup)

bs4.BeautifulSoup

In [3]:
My_table = soup.find("table",{"class":"wikitable sortable tpl-blanktable mw-collapsible"})

In [4]:
import pandas as pd
df=pd.read_html(str(My_table))
# convert list to dataframe
df=pd.DataFrame(df[0])


In [5]:
df.columns

MultiIndex([(  'Polling firm',        'Polling firm'),
            ('Fieldwork date',      'Fieldwork date'),
            (    'Samplesize',          'Samplesize'),
            (          'Abs.',                'Abs.'),
            (         'Union',  'Unnamed: 4_level_1'),
            (           'SPD',  'Unnamed: 5_level_1'),
            (           'AfD',  'Unnamed: 6_level_1'),
            (           'FDP',  'Unnamed: 7_level_1'),
            (         'Linke',  'Unnamed: 8_level_1'),
            (         'Grüne',  'Unnamed: 9_level_1'),
            (            'FW', 'Unnamed: 10_level_1'),
            (        'Others',              'Others'),
            (          'Lead',                'Lead')],
           )

In [6]:
df.columns = [x[0] for x in df.columns]

In [7]:
df.tail()

,Polling firm,Fieldwork date,Samplesize,Abs.,Union,SPD,AfD,FDP,Linke,Grüne,FW,Others,Lead
170,Infratest dimap,4–6 Jan 2021,1520,–,35.0,14.0,10.0,7.0,7.0,21.0,–,6.0,14
171,YouGov,30 Dec 2020–5 Jan 2021,1588,–,36.0,15.0,10.0,6.0,9.0,18.0,–,6.0,18
172,INSA,1–4 Jan 2021,2072,–,36.0,15.0,11.0,7.5,7.5,18.0,–,5.0,18
173,GMS,29 Dec 2020–4 Jan 2021,1004,–,37.0,16.0,9.0,6.0,8.0,18.0,–,6.0,19
174,2017 federal election,24 Sep 2017,–,23.8,32.9,20.5,12.6,10.7,9.2,8.9,1.0,4.0,12.4


In [8]:
dates = list(df["Fieldwork date"])

In [33]:
months_dict = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", 
          "May": "05", "Jun": "06", "Jul": "07", "Aug": "08", 
          "Sep": "09", "Okt": "10", "Nov": "11", "Dec": "12"}

In [10]:
import re
temp = []
for c in dates:
    temp.append(re.split("–| ", c))


In [34]:
days_dict = {"Jan": "31", "Feb": "28", "Mar": "31", "Apr": "30",
        "May": "31", "Jun": "30", "Jul": "31", "Aug": "31", 
        "Sep": "30", "Okt": "31", "Nov": "30", "Dec": "31"}

In [35]:

def poll_to_year(year_dataframe):
    year = []
#iterate over list of poll timeframes each one being a list
    for poll in year_dataframe:
        dates = []
        month = []

        # iterate over each poll date splitted 
        for element in poll:

            # check if element is a date 
            if len(element) == 2 or len(element) == 1:
                dates.append(int(element))


            # check if element is a month
            if len(element) == 3:
                month.append(element)


        # if poll was conducted on one single day
        if len(dates) == 1:
            #append poll day to year list as a list through split()
            year.append(f"{dates[0]}/{months_dict[month[0]]}/2021".split())
            
        
        else:
            # try to get all dates on which the poll was conducted
            x = list(range(dates[0], (int(dates[1]) + 1)))


            sub_list = []



            # check if x is valid range 
            if x:

                #iterate over alls days and append them as a list with theirs month as digits and the year to a normal dd/mm/yyyy syntax
                for date in x:
                    sub_list.append(f"{date}/{months_dict[month[0]]}/2021")


                #append list of lists for one poll to the year list
                year.append(sub_list)


            # if the poll was conducted in two month x is  an empty list and thus False ->this will be executed
            else:
                # get the dates of the polls in the first month
                dates_first_month = range(dates[0], (int(days_dict[month[0]]) + 1))

                #append each day in formant dd/mm/yyyy as a list to sub_list
                for date in dates_first_month:
                    sub_list.append(f"{date}/{months_dict[month[0]]}/2021")


                # get the dates of the polls in the second month
                dates_second_month = range(1, (dates[1] +1))

                #append each day in formant dd/mm/yyyy as a list to sub_list                          
                for date in dates_second_month:
                    sub_list.append(f"{date}/{months_dict[month[1]]}/2021")

                #append list of lists to years
                year.append(sub_list)
    
    return year

In [42]:
df.to_csv("2021.csv")

In [41]:
for i, a in enumerate(final):
    print(a, temp[i])

['11/08/2021', '12/08/2021', '13/08/2021', '14/08/2021', '15/08/2021', '16/08/2021', '17/08/2021'] ['11', '17', 'Aug', '2021']
['5/08/2021', '6/08/2021', '7/08/2021', '8/08/2021', '9/08/2021', '10/08/2021', '11/08/2021', '12/08/2021', '13/08/2021', '14/08/2021', '15/08/2021', '16/08/2021', '17/08/2021'] ['5', '17', 'Aug', '2021']
['13/08/2021', '14/08/2021', '15/08/2021', '16/08/2021'] ['13', '16', 'Aug', '2021']
['10/08/2021', '11/08/2021', '12/08/2021', '13/08/2021', '14/08/2021', '15/08/2021', '16/08/2021'] ['10', '16', 'Aug', '2021']
['9/08/2021', '10/08/2021', '11/08/2021', '12/08/2021', '13/08/2021'] ['9', '13', 'Aug', '2021']
['10/08/2021', '11/08/2021', '12/08/2021'] ['10', '12', 'Aug', '2021']
['4/08/2021', '5/08/2021', '6/08/2021', '7/08/2021', '8/08/2021', '9/08/2021', '10/08/2021'] ['4', '10', 'Aug', '2021']
['6/08/2021', '7/08/2021', '8/08/2021', '9/08/2021'] ['6', '9', 'Aug', '2021']
['3/08/2021', '4/08/2021', '5/08/2021', '6/08/2021', '7/08/2021', '8/08/2021', '9/08/2021

In [45]:
zzz = pd.DataFrame(final)

zzz.to_csv("all days.csv")